In [34]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [2]:
import json
with open('/Users/Ivy/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()



dict_keys(['client-id', 'API Key'])

In [3]:
#pip install yelpapi

In [14]:
# import the YelpAPI Class
from yelpapi import YelpAPI
# Create an instance with your key
yelp= YelpAPI(login['API Key'], timeout_s=5.0)
yelp


In [6]:
# set our API call parameters and filename before the first call
location = 'Los Angeles, CA 90001'
term = 'sushi'

In [7]:
FOLDER ='Data/'
os.makedirs(FOLDER,exist_ok=True)

In [8]:
location.split(',')[0]

'Los Angeles'

In [9]:
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [10]:
JSON_FILE

'Data/Los Angeles-sushi.json'

In [12]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    if len(folder)>0:
        os.makedirs(folder, exist_ok=True)
        print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
        with open (JSON_FILE, 'w') as f:
            json.dump([],f)
else:
    print(f"[i]{JSON_FILE} already exists")

[i]Data/Los Angeles-sushi.json already exists


In [15]:
results = yelp.search_query(term = term,location = location)

In [16]:
type(results)

dict

In [17]:
len(results)

3

In [18]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [19]:
results["total"]

4500

In [22]:
results_per_page = len(results['businesses'])
results_per_page

20

In [27]:
# Use math.ceil to round up for the total number of pages of results.
import time, math

n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

225

In [31]:
for i in tqdm_notebook( range(1,n_pages+1)):
    try:
        time.sleep(.2)
        with open (JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        n_results = len(previous_results)
        results = yelp.search_query(location = location, term = term, offest = n_results+1)
        previous_results.extend(results['businesses'])
        with open (JSON_FILE, 'w') as f:
            json.dump(previous_results,f)
    except Exception as e:
        print('[!] ERROR:', e)

  0%|          | 0/225 [00:00<?, ?it/s]

In [36]:
df = pd.read_json(JSON_FILE)

In [37]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8TkMaePsk1-dVqrTJ7Aarw,crazy-rockn-sushi-gardena-gardena-2,Crazy Rock'N Sushi - Gardena,https://s3-media1.fl.yelpcdn.com/bphoto/KgR3UM...,False,https://www.yelp.com/biz/crazy-rockn-sushi-gar...,924,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 33.890149, 'longitude': -118.303588}","[pickup, delivery]",$$,"{'address1': '1546 W Redondo Beach Blvd', 'add...",13103237655,(310) 323-7655,10580.897119
1,InOqSX9rLUAqAP7Q13UrQg,tenno-sushi-los-angeles-3,Tenno Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/nIScE_...,False,https://www.yelp.com/biz/tenno-sushi-los-angel...,2040,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 34.0475658195504, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '207 S Central Ave', 'address2': ...",12136250602,(213) 625-0602,7937.247513
2,us0WnDOySVXXXwCqs0AaCw,izakaya-osen-los-angeles-los-angeles,Izakaya Osen - Los Angeles,https://s3-media1.fl.yelpcdn.com/bphoto/m2GLJb...,False,https://www.yelp.com/biz/izakaya-osen-los-ange...,1543,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 34.08330553596485, 'longitude': -...","[restaurant_reservation, pickup, delivery]",$$,"{'address1': '2903 Sunset Blvd', 'address2': '...",13239282220,(323) 928-2220,11274.145241
3,6DbAYZ3iMqHn60Maepqa9Q,casa-noodle-teriyaki-los-angeles-2,Casa Noodle Teriyaki,https://s3-media2.fl.yelpcdn.com/bphoto/0Wy60l...,False,https://www.yelp.com/biz/casa-noodle-teriyaki-...,160,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 33.98642, 'longitude': -118.27357}","[pickup, delivery]",$,"{'address1': '5930 S Main St', 'address2': Non...",13232328200,(323) 232-8200,501.788497
4,hARUOlpi_buVjoU0gN2ugw,la-zona-sushi-los-angeles,La Zona Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/cbhseL...,False,https://www.yelp.com/biz/la-zona-sushi-los-ang...,65,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 33.9821249031109, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '1913 E Gage Ave', 'address2': ''...",13234875951,(323) 487-5951,3197.567803


In [38]:
csv_file = JSON_FILE.replace('.json', '.csv.gz')
csv_file

'Data/Los Angeles-sushi.csv.gz'

In [39]:
df.to_csv(csv_file, compression = 'gzip', index = False)